In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('C:/Users/asus/Documents/2.csv', usecols=['content'])#mention the name of file to be extracted
df

,content
0,"It stopped working, it loads for a little bit...."
1,I've not been able to run this app so my revie...
2,It's surprising how bad the app is compared to...
3,"A colossal waste of time. Somehow, the prompts..."
4,It looks cool but it just doesn't work on my p...
...,...
55817,👍👍
55818,😍😍😍
55819,💙💙💙💙💙
55820,👍👍👍👍


In [2]:
df['totalwords'] = df['content'].str.split().str.len() # Calculate word count

In [3]:
data = df.loc[df['totalwords'] > 5] #Removing all reviews having less than two words

In [4]:
data = data.drop('totalwords',axis = 1)

In [5]:
df = data[data['content'].map(lambda x: x.isascii())] #Removing all non-english reviews

In [6]:
df = df.dropna(how='all') # dropping blank rows

In [7]:
df.to_csv("New.csv")

In [8]:
df

,content
0,"It stopped working, it loads for a little bit...."
1,I've not been able to run this app so my revie...
2,It's surprising how bad the app is compared to...
3,"A colossal waste of time. Somehow, the prompts..."
4,It looks cool but it just doesn't work on my p...
...,...
55078,Me a ayudado mucho y si bien la calidad de res...
55079,Muy bien!!! Va igual de bien que en la web!!!
55089,Kya batae fan ho geye iska
55135,Chat GPT menurut saya adalah AI terbaik sejauh...


In [9]:
xx = df['content'].to_string(header=False, index=False)

In [10]:
import re


def remove_url(text):
    text = re.sub(r"http\S+", "", text)
    return text


# Example string with weird font characters plus an URL which we gonna remove.
sample =  xx

sample = remove_url(sample)
#print(f"Text after removing url:- \n {sample}")

In [11]:
!pip install demoji
import demoji


def handle_emoji(string):
    emojis = demoji.findall(string)

    for emoji in emojis:
        string = string.replace(emoji, " " + emojis[emoji].split(":")[0])

    return string


#print(f"Before Handling emoji:- \n {sample}")
#print(f"After Handling emoji:- \n {handle_emoji(sample)}")


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
def word_tokenizer(text):
    text = text.lower()
    text = text.split()

    return text


sample = sample
#print(sample)
#print(word_tokenizer(sample))

In [13]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
en_stopwords = set(stopwords.words('english'))
en_stopwords.update(('chatgpt','chatgpt ','CHATGPT'))#Words to be ommitted 
#print(f"Stop Words in English : \n{ en_stopwords}")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def remove_stopwords(text):
    text = [word for word in text if word not in en_stopwords]
    
    return text


#print(f"Before removing stopwords : {word_tokenizer(sample)}")
#print(f"After removing stopwords : {remove_stopwords(word_tokenizer(sample))}")

In [15]:
def lemmatization(text): #TO CONVERT WORDS IN THEIR BASE FORM IE RAN, RUNNIN, RUN TO RUN

    # text = [sp(word).lemma_ for word in text]

    text = " ".join(text)
    token = sp(text)

    text = [word.lemma_ for word in token]
    return text


#print(f"Before Lemmatization : {word_tokenizer(sample)}")
#print(f"After Lemmatization : {lemmatization(word_tokenizer(sample))}")

In [16]:
def preprocessing(text):
    
    text = remove_url(text) 
    text = handle_emoji(text)
    text = text.lower() 
    text = re.sub(r'[^\w\s]', '', text)
    text = word_tokenizer(text)
    # text = stemming(text)
    text = lemmatization(text)
    text = remove_stopwords(text)
    text = " ".join(text)

    return text

In [17]:
import spacy
sp = spacy.load("en_core_web_sm")

In [19]:
from tqdm import tqdm

tqdm.pandas()

df['content'] = df['content'].progress_map(preprocessing)

100%|███████████████████████████████████████████████████████████████████████████| 25710/25710 [02:44<00:00, 156.56it/s]


In [20]:
df.to_csv("gpt.csv") 

In [21]:
import gensim #Topic Modelling Technicque called LDA (Latent Dirichlet Allocation)
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = df['content']
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['stop', 'work', 'load', 'little', 'bit', 'soon', 'start', 'type', 'start', 'load', 'know', 'circle', 'turn', 'type', 'box', 'go', 'away', 'completely', 'unusable', 'type', 'anything', 'anymore']


In [22]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=50, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [23]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['chatgpt'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
!pip install spacy
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ','ADV'])
print(data_lemmatized[:1])


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


[['work', 'load', 'little', 'bit', 'soon', 'type', 'start', 'load', 'type', 'box', 'away', 'completely', 'unusable', 'type', 'anymore']]


In [25]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])

[[(0, 1), (2, 1), (3, 1), (4, 1), (6, 1), (9, 1), (10, 2), (11, 1), (12, 1), (15, 3), (16, 1), (17, 1)]]


In [26]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [27]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 3, #change it 1-10
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           per_word_topics=True)

In [28]:
from gensim.models import CoherenceModel
# Compute Coherence Score for topic 2
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.367924299691438


## from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [32]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts= data_lemmatized)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 3 elements

In [147]:
df_dominant_topic.to_csv("Q2 Single words Group.csv")

In [30]:
from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_lemmatized for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

In [31]:
df.to_csv("Q2 Groups Final.csv")